# Predicting Molecular Properties

In [284]:
using CSV
using LinearAlgebra
using DataFrames

In [285]:
using Statistics

In [286]:
using MLDataUtils

In [262]:
using Flux
using Base.Iterators: repeated
using Flux: throttle
using Flux: onehotbatch

In [263]:
using Flux.Tracker, Statistics, DelimitedFiles
using Flux.Tracker: Params, gradient, update!

In [268]:
using Random

### The datasets of this project

In [8]:
readdir("./champs-scalar-coupling/")

10-element Array{String,1}:
 "dipole_moments.csv"               
 "magnetic_shielding_tensors.csv"   
 "mulliken_charges.csv"             
 "potential_energy.csv"             
 "sample_submission.csv"            
 "scalar_coupling_contributions.csv"
 "structures.csv"                   
 "structures.zip"                   
 "test.csv"                         
 "train.csv"                        

### We will use the main files to do the machine learning

In [9]:
train = CSV.read("./champs-scalar-coupling/train.csv");
test = CSV.read("./champs-scalar-coupling/test.csv");
structures = CSV.read("./champs-scalar-coupling/structures.csv");

### 1. Structures of each dataset 

In [8]:
println("The size of train : $(size(train))")
println("The size of test : $(size(test))")
println("The size of structures: $(size(structures))")
println("There are $(length(unique(train[:,1])))")

The size of train : (4658147, 6)
The size of test : (2505542, 5)
The size of structures: (2358657, 6)
There are 4658147


#### train.csv - the training set, where the first column (molecule_name) is the name of the molecule where the coupling constant originates (the corresponding XYZ file is located at ./structures/.xyz), the second (atom_index_0) and third column (atom_index_1) is the atom indices of the atom-pair creating the coupling and the fourth column (scalar_coupling_constant) is the scalar coupling constant that we want to be able to predict

In [10]:
train[1:5,:]

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
,Int64,String,Int64,Int64,String,Float64
1,0,dsgdb9nsd_000001,1,0,1JHC,84.8076
2,1,dsgdb9nsd_000001,1,2,2JHH,-11.257
3,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548
4,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543
5,4,dsgdb9nsd_000001,2,0,1JHC,84.8074


In [11]:
names(train)

6-element Array{Symbol,1}:
 :id                      
 :molecule_name           
 :atom_index_0            
 :atom_index_1            
 :type                    
 :scalar_coupling_constant

### columns_oneHot will store the columns' names that we will translate into one hot batch

In [12]:
function addColumns(columns_A,columns_B,ls)
    """
    add the columns of columns_B in ls to the columns_A
    """
    
    for i in ls
        push!(columns_A,columns_B[i])
    end
end
        

addColumns (generic function with 1 method)

In [287]:
columns_oneHot = []

0-element Array{Any,1}

### test.csv - the test set; same info as train, without the target variable: scalar_coupling_constant. In this project we will not use this test dataset to test dataset. We just split the trian dataset into train and test to check our model.

In [288]:
train[1:5,:]

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
,Int64,String,Int64,Int64,String,Float64
1,0,dsgdb9nsd_000001,1,0,1JHC,84.8076
2,1,dsgdb9nsd_000001,1,2,2JHH,-11.257
3,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548
4,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543
5,4,dsgdb9nsd_000001,2,0,1JHC,84.8074


#### structures.csv - containing molecular structure , where the first column is molecule_name, the second column(atom_index) is the name of the molecule the number of atoms in the molecule, followed by a column(atom) containing the atomic element (H for hydrogen, C for carbon etc.) and the remaining columns contain the X, Y and Z cartesian coordinates (a standard format for chemists and molecular visualization programs)

In [14]:
structures[1:5,:]

,molecule_name,atom_index,atom,x,y,z
,String,Int64,String,Float64,Float64,Float64
1,dsgdb9nsd_000001,0,C,-0.0126981,1.0858,0.008001
2,dsgdb9nsd_000001,1,H,0.00215042,-0.00603132,0.00197612
3,dsgdb9nsd_000001,2,H,1.01173,1.46375,0.000276575
4,dsgdb9nsd_000001,3,H,-0.540815,1.44753,-0.876644
5,dsgdb9nsd_000001,4,H,-0.523814,1.43793,0.906397


In [15]:
println("There are $(length(unique(train[:,2]))) distinct molecules in train data")
println("There are $(length(unique(test[:,2]))) distinct molecules in test data")
println("There are $(length(unique(structures[:,2]))) distinct molecules in structures data")
println("There are $(length(unique(structures[:,3]))) distinct atoms in structures data")
println("There are $(length(unique(train[:,5]))) distinct types in train data")


There are 85003 distinct molecules in train data
There are 45772 distinct molecules in test data
There are 29 distinct molecules in structures data
There are 5 distinct atoms in structures data
There are 8 distinct types in train data


### 2. Define a function to combine two dataframes into a big one dataframe and change the columns' names accordingly. And at last construct an complete dataframe containing the essential parameters

In [289]:
function mergeDF(df1,df2,on,atom_index="false")
    """
    parameters:
    df1: a dataframe
    df2: a dataframe
    on: list of tuples containing the columns in df1 and df2. example: (:column1,:column2),column1 of df1 correspond to column2 of df2
    atom_index: if not "false" then the columns in the combined dataframe df3 should be add to the index, otherwise just return the df3
    
    Output: 
    Dataframe df3 consists of df1 and df2 
    """
    df3 = join(df1,df2,on=on)
    
    if atom_index != "false" # should not use the bool value false, as integer 0 is equla to false
        
        add_columns = setdiff(names(df3),names(df1))
        add_columns = [Symbol(x,"_", string(atom_index)) for x in add_columns]
        changed_columns = vcat(names(df1),add_columns)
        
        rename!(df3,names(df3) .=> changed_columns)# rename the column names
        return df3
    else
        return df3
    end
end
    
    

mergeDF (generic function with 2 methods)

### merge the train and structures dataframes and add the columns of structures dataframe according to the column atom_index_0 of train dataframe. The out dataframe is the combination of two dataframes represented by df_2_1.

In [290]:
df_2_1= mergeDF(train,structures,[(:molecule_name, :molecule_name), (:atom_index_0, :atom_index)],0);

In [291]:
df_2_1[1:5,:]

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
,Int64,String,Int64,Int64,String,Float64
1,0,dsgdb9nsd_000001,1,0,1JHC,84.8076
2,1,dsgdb9nsd_000001,1,2,2JHH,-11.257
3,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548
4,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543
5,4,dsgdb9nsd_000001,2,0,1JHC,84.8074


In [17]:
names(df_2_1)

10-element Array{Symbol,1}:
 :id                      
 :molecule_name           
 :atom_index_0            
 :atom_index_1            
 :type                    
 :scalar_coupling_constant
 :atom_0                  
 :x_0                     
 :y_0                     
 :z_0                     

### merge the df_2_1 and structures dataframes by adding the columns of structures dataframe according to the column atom_index_1 of train dataframe

In [18]:
df_2_2 = mergeDF(df_2_1,structures,[(:molecule_name, :molecule_name), (:atom_index_1, :atom_index)],1);

In [292]:
println("The size of df_2_2 is $(size(df_2_2))")

The size of df_2_2 is (4658147, 14)


In [293]:
names(df_2_2)

14-element Array{Symbol,1}:
 :id                      
 :molecule_name           
 :atom_index_0            
 :atom_index_1            
 :type                    
 :scalar_coupling_constant
 :atom_0                  
 :x_0                     
 :y_0                     
 :z_0                     
 :atom_1                  
 :x_1                     
 :y_1                     
 :z_1                     

In [295]:
addColumns(columns_oneHot,names(df_2_2),[3,4,5,7,11])

In [296]:
columns_oneHot

5-element Array{Any,1}:
 :atom_index_0
 :atom_index_1
 :type        
 :atom_0      
 :atom_1      

### We will use df_2_2 as the training data. Now we have added structures parameters to the dataframe. And  we will  use Machine learning to check whether these parameters could produce an acceptable model.

#### delete_columns contains the columns that will be deleted from df_2_2

In [23]:
delete_columns = [:id,:molecule_name,:scalar_coupling_constant];

In [24]:
X = select(df_2_2,Not(delete_columns))

,atom_index_0,atom_index_1,type,atom_0,x_0,y_0,z_0
,Int64,Int64,String,String,Float64,Float64,Float64
1,1,0,1JHC,H,0.00215042,-0.00603132,0.00197612
2,1,2,2JHH,H,0.00215042,-0.00603132,0.00197612
3,1,3,2JHH,H,0.00215042,-0.00603132,0.00197612
4,1,4,2JHH,H,0.00215042,-0.00603132,0.00197612
5,2,0,1JHC,H,1.01173,1.46375,0.000276575
6,2,3,2JHH,H,1.01173,1.46375,0.000276575
7,2,4,2JHH,H,1.01173,1.46375,0.000276575
8,3,0,1JHC,H,-0.540815,1.44753,-0.876644
9,3,4,2JHH,H,-0.540815,1.44753,-0.876644


### translate the columns into one hot batch

In [26]:
function hotlize(df,column_name)
    cate = unique(df[:,column_name])
    lables = df[:,column_name]
    hot = onehotbatch(lables,cate)
    return hot
end

hotlize (generic function with 1 method)

In [299]:
@timev hot1 = hotlize(X,columns_oneHot[1])

  0.541678 seconds (27 allocations: 106.619 MiB, 24.97% gc time)
elapsed time (ns): 541678423
gc time (ns):      135250412
bytes allocated:   111798240
pool allocs:       24
malloc() calls:    3
GC pauses:         3


29×4658147 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 1  1  1  1  0  0  0  0  0  0  1  1  1  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  1  1  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  1  1  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0 

In [300]:
for i in columns_oneHot[2:end]
    hot1 = vcat(hot1,hotlize(X,i))
end

In [301]:
hot1

70×4658147 Array{Bool,2}:
 1  1  1  1  0  0  0  0  0  0  1  1  1  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  1  1  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  1  1  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0

In [302]:
Y_train = df_2_2[:scalar_coupling_constant]

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[302]:1
└ @ Core In[302]:1


4658147-element Array{Float64,1}:
  84.8076  
 -11.257   
 -11.2548  
 -11.2543  
  84.8074  
 -11.2541  
 -11.2548  
  84.8093  
 -11.2543  
  84.8095  
  32.6889  
 -11.1866  
 -11.1757  
   ⋮       
   1.02549 
  99.6572  
   9.11973 
   0.789559
  -0.006537
   1.94438 
   0.861412
   3.54345 
   0.568997
   1.17337 
   4.76201 
 117.934   

#### X_remain contains the columns besides the columns in columns_oneHot

In [31]:
X_remain = select(X,Not(columns_oneHot))

,x_0,y_0,z_0,x_1,y_1,z_1
,Float64,Float64,Float64,Float64,Float64,Float64
1,0.00215042,-0.00603132,0.00197612,-0.0126981,1.0858,0.008001
2,0.00215042,-0.00603132,0.00197612,1.01173,1.46375,0.000276575
3,0.00215042,-0.00603132,0.00197612,-0.540815,1.44753,-0.876644
4,0.00215042,-0.00603132,0.00197612,-0.523814,1.43793,0.906397
5,1.01173,1.46375,0.000276575,-0.0126981,1.0858,0.008001
6,1.01173,1.46375,0.000276575,-0.540815,1.44753,-0.876644
7,1.01173,1.46375,0.000276575,-0.523814,1.43793,0.906397
8,-0.540815,1.44753,-0.876644,-0.0126981,1.0858,0.008001
9,-0.540815,1.44753,-0.876644,-0.523814,1.43793,0.906397


### Convert the X_remain to a matrix

In [32]:
M_X = convert(Matrix,X_remain)

4658147×6 Array{Float64,2}:
  0.00215042  -0.00603132   0.00197612   -0.0126981   1.0858      0.008001   
  0.00215042  -0.00603132   0.00197612    1.01173     1.46375     0.000276575
  0.00215042  -0.00603132   0.00197612   -0.540815    1.44753    -0.876644   
  0.00215042  -0.00603132   0.00197612   -0.523814    1.43793     0.906397   
  1.01173      1.46375      0.000276575  -0.0126981   1.0858      0.008001   
  1.01173      1.46375      0.000276575  -0.540815    1.44753    -0.876644   
  1.01173      1.46375      0.000276575  -0.523814    1.43793     0.906397   
 -0.540815     1.44753     -0.876644     -0.0126981   1.0858      0.008001   
 -0.540815     1.44753     -0.876644     -0.523814    1.43793     0.906397   
 -0.523814     1.43793      0.906397     -0.0126981   1.0858      0.008001   
  0.0172575    0.0125452   -0.0273772    -0.0404261   1.02411     0.0625638  
  0.0172575    0.0125452   -0.0273772     0.915789    1.35875    -0.0287578  
  0.0172575    0.0125452   -0.027377

### Transpose the matrix M_X

In [224]:
MT_X = M_X'

6×4658147 Adjoint{Float64,Array{Float64,2}}:
  0.00215042   0.00215042    0.00215042  …   1.12655     1.12655    1.12655 
 -0.00603132  -0.00603132   -0.00603132     -1.34873    -1.34873   -1.34873 
  0.00197612   0.00197612    0.00197612     -1.93384    -1.93384   -1.93384 
 -0.0126981    1.01173      -0.540815       -0.0270757  -0.131901   0.787756
  1.0858       1.46375       1.44753         0.747033    0.356983  -0.840138
  0.008001     0.000276575  -0.876644    …   0.478506   -1.0102    -1.04215 

### combine the MT_X and hot1 matrix. The overall matrix(x) is the data we will use to train the model

In [34]:
x = vcat(hot1,MT_X)

76×4658147 Array{Float64,2}:
  1.0          1.0           1.0         …   0.0         0.0        0.0     
  0.0          0.0           0.0             0.0         0.0        0.0     
  0.0          0.0           0.0             0.0         0.0        0.0     
  0.0          0.0           0.0             0.0         0.0        0.0     
  0.0          0.0           0.0             0.0         0.0        0.0     
  0.0          0.0           0.0         …   0.0         0.0        0.0     
  0.0          0.0           0.0             0.0         0.0        0.0     
  0.0          0.0           0.0             0.0         0.0        0.0     
  0.0          0.0           0.0             0.0         0.0        0.0     
  0.0          0.0           0.0             0.0         0.0        0.0     
  0.0          0.0           0.0         …   0.0         0.0        0.0     
  0.0          0.0           0.0             0.0         0.0        0.0     
  0.0          0.0           0.0             0.

### The target value y

In [306]:
y = Y_train'

1×4658147 Adjoint{Float64,Array{Float64,1}}:
 84.8076  -11.257  -11.2548  -11.2543  …  0.568997  1.17337  4.76201  117.934

### Spliting the data into (x_train,y_train), (x_test,y_test)

In [156]:
Xs, Ys = shuffleobs((x, y))

([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; -0.8907004178 6.813941901 … -0.6331429994 -0.777584271; -0.9283688187 0.3250844165 … -1.360241962 1.275119252], [8.1091 -1.013 … 88.0636 -1.59181])

In [157]:
(x_train,y_train), (x_test,y_test) = splitobs((Xs, Ys); at = 0.1)

(([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; -0.8907004178 6.813941901 … -0.0454236917 3.273593606; -0.9283688187 0.3250844165 … -2.155780183 -0.4422880514], [8.1091 -1.013 … 0.697691 8.27806]), ([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.1994557395 -0.4635824252 … -0.6331429994 -0.777584271; -0.948272546 2.349988992 … -1.360241962 1.275119252], [1.95463 0.198646 … 88.0636 -1.59181]))

In [158]:
size(x_train)

(76, 465815)

### The model 1

In [163]:
m = Chain(
    Dense(76,1),
      
)

Chain(Dense(76, 1))

In [164]:
loss(x, y) = Flux.mse(m(x_train), y_train)

loss (generic function with 1 method)

In [166]:
dataset = repeated((x_train, y_train), 20)
evalcb = () -> @show(loss(x_train, y_train))
opt = ADAM()

ADAM(0.001, (0.9, 0.999), IdDict{Any,Any}())

In [167]:
Flux.train!(loss, params(m), dataset, opt, cb = throttle(evalcb, 2))

loss(x_train, y_train) = 1477.0165649124863 (tracked)
loss(x_train, y_train) = 1475.8624892874047 (tracked)
loss(x_train, y_train) = 1474.7116969436463 (tracked)
loss(x_train, y_train) = 1473.5644123348882 (tracked)


### we will see that the learning rate is too slow when learning rate is 0.001. So we will change the parameter learning rate to 0.2

In [168]:
opt = ADAM(0.2)

ADAM(0.2, (0.9, 0.999), IdDict{Any,Any}())

In [169]:
Flux.train!(loss, params(m), dataset, opt, cb = throttle(evalcb, 2))

loss(x_train, y_train) = 1429.2387350407407 (tracked)
loss(x_train, y_train) = 1296.2363468343292 (tracked)
loss(x_train, y_train) = 1195.6178266401696 (tracked)
loss(x_train, y_train) = 1091.3735784155167 (tracked)
loss(x_train, y_train) = 1012.1967266888711 (tracked)


### That's much better. How about changing the learning rate to 0.5

In [171]:
opt = ADAM(0.5)

ADAM(0.5, (0.9, 0.999), IdDict{Any,Any}())

In [172]:
Flux.train!(loss, params(m), dataset, opt, cb = throttle(evalcb, 2))

loss(x_train, y_train) = 966.6196634607235 (tracked)
loss(x_train, y_train) = 834.6004407233077 (tracked)
loss(x_train, y_train) = 728.0746106716643 (tracked)
loss(x_train, y_train) = 637.2567203455269 (tracked)


### Much better. In this problem, mybe we should set a high learning rate at first, and then set a low learning rate at the end of process. Next we will change the following parameters: dataset = repeated((x_train, y_train), 100), and the learning rate = 0.1

In [173]:
dataset = repeated((x_train, y_train), 100)
opt = ADAM(0.1)

ADAM(0.1, (0.9, 0.999), IdDict{Any,Any}())

In [174]:
Flux.train!(loss, params(m), dataset, opt, cb = throttle(evalcb, 2))

loss(x_train, y_train) = 570.4828376743883 (tracked)
loss(x_train, y_train) = 557.2269766707021 (tracked)
loss(x_train, y_train) = 541.5115158687368 (tracked)
loss(x_train, y_train) = 526.3296816079801 (tracked)
loss(x_train, y_train) = 511.6144914700099 (tracked)
loss(x_train, y_train) = 497.35314133362084 (tracked)
loss(x_train, y_train) = 483.53241318685207 (tracked)
loss(x_train, y_train) = 470.10537193048697 (tracked)
loss(x_train, y_train) = 457.0383508228054 (tracked)
loss(x_train, y_train) = 444.3140884307231 (tracked)
loss(x_train, y_train) = 431.91514194161687 (tracked)
loss(x_train, y_train) = 419.825626147481 (tracked)
loss(x_train, y_train) = 408.0379878731446 (tracked)
loss(x_train, y_train) = 396.54705073673495 (tracked)
loss(x_train, y_train) = 385.34888724881745 (tracked)
loss(x_train, y_train) = 374.44072969405494 (tracked)
loss(x_train, y_train) = 363.81857680283787 (tracked)
loss(x_train, y_train) = 353.4777625238747 (tracked)
loss(x_train, y_train) = 345.4044083154

### Set the learning rate to 0.3

In [175]:
opt = ADAM(0.3)

ADAM(0.3, (0.9, 0.999), IdDict{Any,Any}())

In [176]:
Flux.train!(loss, params(m), dataset, opt, cb = throttle(evalcb, 2))

loss(x_train, y_train) = 322.00121252091174 (tracked)
loss(x_train, y_train) = 287.22376250954795 (tracked)
loss(x_train, y_train) = 258.3510357330749 (tracked)
loss(x_train, y_train) = 231.83193480990994 (tracked)
loss(x_train, y_train) = 207.58408690172362 (tracked)
loss(x_train, y_train) = 185.69113229969312 (tracked)
loss(x_train, y_train) = 166.26447027891876 (tracked)
loss(x_train, y_train) = 149.17266267557213 (tracked)
loss(x_train, y_train) = 134.25413269243015 (tracked)
loss(x_train, y_train) = 121.35342347611018 (tracked)
loss(x_train, y_train) = 110.29895708287255 (tracked)
loss(x_train, y_train) = 100.91437068771657 (tracked)
loss(x_train, y_train) = 93.02714421125683 (tracked)
loss(x_train, y_train) = 86.46552139522369 (tracked)
loss(x_train, y_train) = 81.06146443281605 (tracked)
loss(x_train, y_train) = 76.6561105174408 (tracked)
loss(x_train, y_train) = 73.7513878380346 (tracked)
loss(x_train, y_train) = 70.78212175435377 (tracked)
loss(x_train, y_train) = 68.433900846

### Check the model

### training data

In [272]:
train_check = m(x_train[:,1:10])

Tracked 1×10 Array{Float32,2}:
 4.59979  3.16413  7.17627  1.67221  …  2.30562  2.3314  91.2066  4.44909

In [178]:
y_train[:,1:10]

1×10 Array{Float64,2}:
 8.1091  -1.013  8.06721  -1.20206  …  -1.0258  -0.415738  103.367  1.6762

In [278]:
for i in zip(train_check',y_train[1:10])
    println(i)
end

(4.5997868f0 (tracked), 8.1091)
(3.164126f0 (tracked), -1.013)
(7.1762743f0 (tracked), 8.06721)
(1.6722097f0 (tracked), -1.20206)
(4.7480025f0 (tracked), -0.164238)
(3.7005816f0 (tracked), 1.80993)
(2.3056245f0 (tracked), -1.0258)
(2.331396f0 (tracked), -0.415738)
(91.20658f0 (tracked), 103.367)
(4.449092f0 (tracked), 1.6762)


### testing data

In [274]:
test_check = m(x_test[:,1:10])

Tracked 1×10 Array{Float32,2}:
 5.57158  6.60521  5.96639  1.35686  …  1.42906  1.29768  91.4144  4.22793

In [181]:
y_test[:,1:10]

1×10 Array{Float64,2}:
 1.95463  0.198646  3.1511  -1.81079  …  -1.00772  91.2698  0.899873

In [279]:
for i in zip(test_check',y_test[1:10])
    println(i)
end

(5.571581f0 (tracked), 1.95463)
(6.6052094f0 (tracked), 0.198646)
(5.966392f0 (tracked), 3.1511)
(1.3568573f0 (tracked), -1.81079)
(5.3730454f0 (tracked), 0.839303)
(4.6628094f0 (tracked), 1.04344)
(1.42906f0 (tracked), 5.62805)
(1.297679f0 (tracked), -1.00772)
(91.41441f0 (tracked), 91.2698)
(4.2279286f0 (tracked), 0.899873)


### From the above result, we can see that the higher the value is, the better result we can get from the model.

## Next, we will add more parameters to the dataframe. And check whether more parameters could create a better model.

### Display other files information

In [182]:
dipole_moments =  CSV.read("./champs-scalar-coupling/dipole_moments.csv");
magnetic_shielding_tensors = CSV.read("./champs-scalar-coupling/magnetic_shielding_tensors.csv");
mulliken_charges = CSV.read("./champs-scalar-coupling/mulliken_charges.csv");
potential_energy = CSV.read("./champs-scalar-coupling/potential_energy.csv");
scalar_coupling_contributions = CSV.read("./champs-scalar-coupling/scalar_coupling_contributions.csv");

### df_3 consist of train, structures, dipole_moments dataframes

In [183]:
println("the size of dipole_moments: $(size(dipole_moments))")
dipole_moments[1:5,:]

the size of dipole_moments: (85003, 4)


,molecule_name,X,Y,Z
,String,Float64,Float64,Float64
1,dsgdb9nsd_000001,0.0,0.0,0.0
2,dsgdb9nsd_000002,-0.0002,0.0,1.6256
3,dsgdb9nsd_000003,0.0,0.0,-1.8511
4,dsgdb9nsd_000005,0.0,0.0,-2.8937
5,dsgdb9nsd_000007,0.0,0.0,0.0


In [185]:
df_3 = mergeDF(df_2_2,dipole_moments,[(:molecule_name,:molecule_name)],"dipole_moments");

In [186]:
println("The size of df3 is $(size(df_3))")

The size of df3 is (4658147, 17)


In [307]:
names(df_3)

17-element Array{Symbol,1}:
 :id                      
 :molecule_name           
 :atom_index_0            
 :atom_index_1            
 :type                    
 :scalar_coupling_constant
 :atom_0                  
 :x_0                     
 :y_0                     
 :z_0                     
 :atom_1                  
 :x_1                     
 :y_1                     
 :z_1                     
 :X_dipole_moments        
 :Y_dipole_moments        
 :Z_dipole_moments        

### df_4 consist of train, structures, dipole_moments, mulliken_charges dataframes

In [188]:
println("the size of mulliken_charges: $(size(mulliken_charges))")
mulliken_charges[1:5,:]

the size of mulliken_charges: (1533537, 3)


,molecule_name,atom_index,mulliken_charge
,String,Int64,Float64
1,dsgdb9nsd_000001,0,-0.535689
2,dsgdb9nsd_000001,1,0.133921
3,dsgdb9nsd_000001,2,0.133922
4,dsgdb9nsd_000001,3,0.133923
5,dsgdb9nsd_000001,4,0.133923


In [189]:
df_4_1 = mergeDF(df_3,mulliken_charges,[(:molecule_name,:molecule_name),(:atom_index_0,:atom_index)],0);

In [190]:
println("The size of df_4_1 is : $(size(df_4_1))")

The size of df_4_1 is : (4658147, 18)


In [191]:
df_4_2 = mergeDF(df_4_1,mulliken_charges,[(:molecule_name,:molecule_name),(:atom_index_1,:atom_index)],1);

In [192]:
println("The size of df_4_2 is : $(size(df_4_2))")

The size of df_4_2 is : (4658147, 19)


In [193]:
names(df_4_2)

19-element Array{Symbol,1}:
 :id                      
 :molecule_name           
 :atom_index_0            
 :atom_index_1            
 :type                    
 :scalar_coupling_constant
 :atom_0                  
 :x_0                     
 :y_0                     
 :z_0                     
 :atom_1                  
 :x_1                     
 :y_1                     
 :z_1                     
 :X_dipole_moments        
 :Y_dipole_moments        
 :Z_dipole_moments        
 :mulliken_charge_0       
 :mulliken_charge_1       

### df_5 consist of train, structures, dipole_moments, mulliken_charges, potential_energy dataframes

In [194]:
println("the size of potential_energy: $(size(potential_energy))")
potential_energy[1:5,:]

the size of potential_energy: (85003, 2)


,molecule_name,potential_energy
,String,Float64
1,dsgdb9nsd_000001,-40.5237
2,dsgdb9nsd_000002,-56.5603
3,dsgdb9nsd_000003,-76.4261
4,dsgdb9nsd_000005,-93.4285
5,dsgdb9nsd_000007,-79.8387


In [195]:
df_5 = mergeDF(df_4_2,potential_energy,[(:molecule_name,:molecule_name)]);

In [196]:
println("The size of df_5 is : $(size(df_5))")

The size of df_5 is : (4658147, 20)


In [197]:
names(df_5)

20-element Array{Symbol,1}:
 :id                      
 :molecule_name           
 :atom_index_0            
 :atom_index_1            
 :type                    
 :scalar_coupling_constant
 :atom_0                  
 :x_0                     
 :y_0                     
 :z_0                     
 :atom_1                  
 :x_1                     
 :y_1                     
 :z_1                     
 :X_dipole_moments        
 :Y_dipole_moments        
 :Z_dipole_moments        
 :mulliken_charge_0       
 :mulliken_charge_1       
 :potential_energy        

### df_6 consist of train, structures, dipole_moments, mulliken_charges, potential_energy, scalar_coupling_contributions dataframes

In [198]:
println("the size of scalar_coupling_contributions: $(size(scalar_coupling_contributions))")
scalar_coupling_contributions[1:5,:]

the size of scalar_coupling_contributions: (4658147, 8)


,molecule_name,atom_index_0,atom_index_1,type,fc,sd,pso
,String,Int64,Int64,String,Float64,Float64,Float64
1,dsgdb9nsd_000001,1,0,1JHC,83.0224,0.254579,1.25862
2,dsgdb9nsd_000001,1,2,2JHH,-11.0347,0.352978,2.85839
3,dsgdb9nsd_000001,1,3,2JHH,-11.0325,0.352944,2.85852
4,dsgdb9nsd_000001,1,4,2JHH,-11.0319,0.352934,2.85855
5,dsgdb9nsd_000001,2,0,1JHC,83.0222,0.254585,1.25861


In [199]:
names(scalar_coupling_contributions)

8-element Array{Symbol,1}:
 :molecule_name
 :atom_index_0 
 :atom_index_1 
 :type         
 :fc           
 :sd           
 :pso          
 :dso          

In [200]:
df_6 = mergeDF(df_5,scalar_coupling_contributions,[(:molecule_name,:molecule_name),(:atom_index_0,:atom_index_0),(:atom_index_1,:atom_index_1),(:type,:type)]);

In [201]:
println("The size of df_6 is : $(size(df_6))")

The size of df_6 is : (4658147, 24)


In [202]:
names(df_6)

24-element Array{Symbol,1}:
 :id                      
 :molecule_name           
 :atom_index_0            
 :atom_index_1            
 :type                    
 :scalar_coupling_constant
 :atom_0                  
 :x_0                     
 :y_0                     
 :z_0                     
 :atom_1                  
 :x_1                     
 :y_1                     
 :z_1                     
 :X_dipole_moments        
 :Y_dipole_moments        
 :Z_dipole_moments        
 :mulliken_charge_0       
 :mulliken_charge_1       
 :potential_energy        
 :fc                      
 :sd                      
 :pso                     
 :dso                     

### df_7 consist of train, structures, dipole_moments, mulliken_charges, potential_energy, scalar_coupling_contributions, magnetic_shielding_tensors dataframes

In [203]:
println("the size of magnetic_shielding_tensors: $(size(magnetic_shielding_tensors))")
magnetic_shielding_tensors[1:5,:]

the size of magnetic_shielding_tensors: (1533537, 11)


,molecule_name,atom_index,XX,YX,ZX,XY,YY,ZY
,String,Int64,Float64,Float64,Float64,Float64,Float64,Float64
1,dsgdb9nsd_000001,0,195.315,0.0,-0.0001,0.0,195.317,0.0007
2,dsgdb9nsd_000001,1,31.341,-1.2317,4.0544,-1.2317,28.9546,-1.7173
3,dsgdb9nsd_000001,2,31.5814,1.2173,-4.1474,1.2173,28.9036,-1.6036
4,dsgdb9nsd_000001,3,31.5172,4.1086,1.2723,4.1088,33.9068,1.695
5,dsgdb9nsd_000001,4,31.4029,-4.0942,-1.1793,-4.0944,34.0776,1.6259


In [204]:
names(magnetic_shielding_tensors)

11-element Array{Symbol,1}:
 :molecule_name
 :atom_index   
 :XX           
 :YX           
 :ZX           
 :XY           
 :YY           
 :ZY           
 :XZ           
 :YZ           
 :ZZ           

In [205]:
df_7_1 = mergeDF(df_6,magnetic_shielding_tensors,[(:molecule_name,:molecule_name),(:atom_index_0,:atom_index)], 0);

In [206]:
df_7_2 = mergeDF(df_7_1,magnetic_shielding_tensors,[(:molecule_name,:molecule_name),(:atom_index_1,:atom_index)], 1);

In [207]:
println("The size of df_7_2 is : $(size(df_7_2))")

The size of df_7_2 is : (4658147, 42)


### 3. df_7_2 is the complete dataframe that contain all the papameters we could add.

In [208]:
Y_complete = df_7_2[:,:scalar_coupling_constant] # target value

4658147-element Array{Float64,1}:
  84.8076  
 -11.257   
 -11.2548  
 -11.2543  
  84.8074  
 -11.2541  
 -11.2548  
  84.8093  
 -11.2543  
  84.8095  
  32.6889  
 -11.1866  
 -11.1757  
   ⋮       
   1.02549 
  99.6572  
   9.11973 
   0.789559
  -0.006537
   1.94438 
   0.861412
   3.54345 
   0.568997
   1.17337 
   4.76201 
 117.934   

In [209]:
y_complete = Y_complete'

1×4658147 Adjoint{Float64,Array{Float64,1}}:
 84.8076  -11.257  -11.2548  -11.2543  …  0.568997  1.17337  4.76201  117.934

### using select method to select the columns in df_7_2 to create the training data X

In [210]:
println("There are $(length(unique(df_7_2[:,:molecule_name]))) unique molecule names in the df_7_2")

There are 85003 unique molecule names in the df_7_2


#### Because there are 85003 unique molecule names in the df_7_2, then 85003 rows will be added to the training data if we use one hot batch. Then the overall matrix will be bigger than 85000×4658147. My omputer can't handle such big matrix. Thus I delete this column in X 

In [213]:
delete_columns = [:id,:molecule_name,:scalar_coupling_constant]

3-element Array{Symbol,1}:
 :id                      
 :molecule_name           
 :scalar_coupling_constant

In [214]:
columns_oneHot 

5-element Array{Any,1}:
 :atom_index_0
 :atom_index_1
 :type        
 :atom_0      
 :atom_1      

In [215]:
X_complete = select(df_7_2,Not(delete_columns))

,atom_index_0,atom_index_1,type,atom_0,x_0,y_0,z_0
,Int64,Int64,String,String,Float64,Float64,Float64
1,1,0,1JHC,H,0.00215042,-0.00603132,0.00197612
2,1,2,2JHH,H,0.00215042,-0.00603132,0.00197612
3,1,3,2JHH,H,0.00215042,-0.00603132,0.00197612
4,1,4,2JHH,H,0.00215042,-0.00603132,0.00197612
5,2,0,1JHC,H,1.01173,1.46375,0.000276575
6,2,3,2JHH,H,1.01173,1.46375,0.000276575
7,2,4,2JHH,H,1.01173,1.46375,0.000276575
8,3,0,1JHC,H,-0.540815,1.44753,-0.876644
9,3,4,2JHH,H,-0.540815,1.44753,-0.876644


## make onehot and delete the associated columns, and change the dataframe to matrix, vcat the hotlized columns. the output is a matrix

### we will create one hot bach based on the following columns

In [144]:
columns_oneHot

5-element Array{Any,1}:
 :atom_index_0
 :atom_index_1
 :type        
 :atom_0      
 :atom_1      

In [218]:
@timev hot2 = hotlize(X_complete,columns_oneHot[1])

  0.141376 seconds (28 allocations: 106.619 MiB, 24.32% gc time)
elapsed time (ns): 141376228
gc time (ns):      34381479
bytes allocated:   111798288
pool allocs:       25
malloc() calls:    3
GC pauses:         3
full collections:  1


29×4658147 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 1  1  1  1  0  0  0  0  0  0  1  1  1  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  1  1  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  1  1  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0 

In [219]:
for i in columns_oneHot[2:end]
    hot2 = vcat(hot2,hotlize(X_complete,i))
end

In [308]:
hot2

70×4658147 Array{Bool,2}:
 1  1  1  1  0  0  0  0  0  0  1  1  1  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  1  1  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  1  1  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0

### delete the columns that have been hotlized in X_complete, and get the X_complete_remain(X_cr for short)

In [221]:
X_cr = select(X_complete,Not(columns_oneHot))

,x_0,y_0,z_0,x_1,y_1,z_1,X_dipole_moments
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.00215042,-0.00603132,0.00197612,-0.0126981,1.0858,0.008001,0.0
2,0.00215042,-0.00603132,0.00197612,1.01173,1.46375,0.000276575,0.0
3,0.00215042,-0.00603132,0.00197612,-0.540815,1.44753,-0.876644,0.0
4,0.00215042,-0.00603132,0.00197612,-0.523814,1.43793,0.906397,0.0
5,1.01173,1.46375,0.000276575,-0.0126981,1.0858,0.008001,0.0
6,1.01173,1.46375,0.000276575,-0.540815,1.44753,-0.876644,0.0
7,1.01173,1.46375,0.000276575,-0.523814,1.43793,0.906397,0.0
8,-0.540815,1.44753,-0.876644,-0.0126981,1.0858,0.008001,0.0
9,-0.540815,1.44753,-0.876644,-0.523814,1.43793,0.906397,0.0


### Convert the X_cr to a matrix as X_m

In [222]:
X_m = convert(Matrix,X_cr)

4658147×34 Array{Float64,2}:
  0.00215042  -0.00603132   0.00197612   …  -0.0001    0.0007  195.317 
  0.00215042  -0.00603132   0.00197612      -4.1476   -1.6036   33.8967
  0.00215042  -0.00603132   0.00197612       1.2724    1.6951   28.9579
  0.00215042  -0.00603132   0.00197612      -1.1795    1.626    28.9013
  1.01173      1.46375      0.000276575     -0.0001    0.0007  195.317 
  1.01173      1.46375      0.000276575  …   1.2724    1.6951   28.9579
  1.01173      1.46375      0.000276575     -1.1795    1.626    28.9013
 -0.540815     1.44753     -0.876644        -0.0001    0.0007  195.317 
 -0.540815     1.44753     -0.876644        -1.1795    1.626    28.9013
 -0.523814     1.43793      0.906397        -0.0001    0.0007  195.317 
  0.0172575    0.0125452   -0.0273772    …   0.0161   -0.0004  237.497 
  0.0172575    0.0125452   -0.0273772       -2.3616    4.1861   27.9885
  0.0172575    0.0125452   -0.0273772        4.8083   -0.0488   27.9889
  ⋮                                

### Transpose the matrix X_m as X_tm(transpose matrix)

In [225]:
X_tm = X_m'

34×4658147 Adjoint{Float64,Array{Float64,2}}:
   0.00215042    0.00215042     0.00215042  …     1.12655      1.12655 
  -0.00603132   -0.00603132    -0.00603132       -1.34873     -1.34873 
   0.00197612    0.00197612     0.00197612       -1.93384     -1.93384 
  -0.0126981     1.01173       -0.540815         -0.131901     0.787756
   1.0858        1.46375        1.44753           0.356983    -0.840138
   0.008001      0.000276575   -0.876644    …    -1.0102      -1.04215 
   0.0           0.0            0.0               1.3623       1.3623  
   0.0           0.0            0.0               1.4058       1.4058  
   0.0           0.0            0.0              -0.0         -0.0     
   0.133921      0.133921       0.133921          0.090746     0.090746
  -0.535689      0.133922       0.133923    …     0.010044    -0.097191
 -40.5237      -40.5237       -40.5237         -364.873     -364.873   
  83.0224      -11.0347       -11.0325            4.80062    115.975   
   ⋮              

### combine the X_tm and hot2 matrix into x_complete. The x_complete matrix is the data we will use to train the model

### The x_complete

In [226]:
x_complete = vcat(hot2,X_tm)

104×4658147 Array{Float64,2}:
   1.0      1.0      1.0      1.0     …    0.0       0.0       0.0   
   0.0      0.0      0.0      0.0          0.0       0.0       0.0   
   0.0      0.0      0.0      0.0          0.0       0.0       0.0   
   0.0      0.0      0.0      0.0          0.0       0.0       0.0   
   0.0      0.0      0.0      0.0          0.0       0.0       0.0   
   0.0      0.0      0.0      0.0     …    0.0       0.0       0.0   
   0.0      0.0      0.0      0.0          0.0       0.0       0.0   
   0.0      0.0      0.0      0.0          0.0       0.0       0.0   
   0.0      0.0      0.0      0.0          0.0       0.0       0.0   
   0.0      0.0      0.0      0.0          0.0       0.0       0.0   
   0.0      0.0      0.0      0.0     …    0.0       0.0       0.0   
   0.0      0.0      0.0      0.0          0.0       0.0       0.0   
   0.0      0.0      0.0      0.0          0.0       0.0       0.0   
   ⋮                                  ⋱              ⋮      

In [227]:
y_complete = Y_complete'

1×4658147 Adjoint{Float64,Array{Float64,1}}:
 84.8076  -11.257  -11.2548  -11.2543  …  0.568997  1.17337  4.76201  117.934

### Split into train and test sets

In [228]:
Xs_complete, Ys_complete = shuffleobs((x_complete, y_complete))

([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 1.9996 -1.4347 … -3.7329 -0.1907; 148.36 24.0246 … 148.997 95.6348], [6.93372 7.0001 … -1.38691 0.419022])

#### xc_train is the x_complete_train. The same as yc_train, xc_test,yc_test

In [229]:
(xc_train,yc_train), (xc_test,yc_test) = splitobs((Xs_complete, Ys_complete); at = 0.1)

(([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 1.9996 -1.4347 … 55.4072 8.4297; 148.36 24.0246 … 199.429 116.606], [6.93372 7.0001 … 0.776652 0.454089]), ([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; -0.0582 38.8408 … -3.7329 -0.1907; 24.7684 71.3407 … 148.997 95.6348], [5.08376 -2.76404 … -1.38691 0.419022]))

In [230]:
size(xc_train)

(104, 465815)

In [231]:
size(yc_train)

(1, 465815)

### shuffleobs and splitobs  do not return a new array, but a lazy view in the form of a SubArray.

In [232]:
typeof(x_train)

SubArray{Float64,2,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},Array{Int64,1}},false}

### The model 2

In [233]:
mc = Chain(
    Dense(104,1),
      
)

Chain(Dense(104, 1))

In [234]:
loss(x, y) = Flux.mse(mc(x), y)

loss (generic function with 1 method)

In [235]:
dataset_complete = repeated((xc_train, yc_train), 20)
evalcb_complete = () -> @show(loss(xc_train, yc_train))
opt_complete = ADAM(0.5)

ADAM(0.5, (0.9, 0.999), IdDict{Any,Any}())

In [236]:
Flux.train!(loss, params(mc), dataset_complete, opt_complete, cb = throttle(evalcb_complete, 2))

loss(xc_train, yc_train) = 154518.01661692932 (tracked)
loss(xc_train, yc_train) = 87061.9217344767 (tracked)
loss(xc_train, yc_train) = 2261.8235294923134 (tracked)
loss(xc_train, yc_train) = 33543.97852311234 (tracked)
loss(xc_train, yc_train) = 8541.98316200719 (tracked)
loss(xc_train, yc_train) = 12707.309208416777 (tracked)
loss(xc_train, yc_train) = 8090.825717732694 (tracked)


### The above result shows that the learning rate is too big. So we change the learning rate to 0.2

In [238]:
opt_complete = ADAM(0.2)

ADAM(0.2, (0.9, 0.999), IdDict{Any,Any}())

In [239]:
Flux.train!(loss, params(mc), dataset_complete, opt_complete, cb = throttle(evalcb_complete, 2))

loss(xc_train, yc_train) = 3233.6782603058823 (tracked)
loss(xc_train, yc_train) = 62.728829850595005 (tracked)
loss(xc_train, yc_train) = 3161.995260085338 (tracked)
loss(xc_train, yc_train) = 2637.865033648555 (tracked)
loss(xc_train, yc_train) = 100.16668410340306 (tracked)
loss(xc_train, yc_train) = 1779.4201388077772 (tracked)
loss(xc_train, yc_train) = 282.16111964426403 (tracked)


### The learning rate is also too big for 0.2. Then we change the learning rate to 0.01

In [243]:
opt_complete = ADAM(0.01)

ADAM(0.01, (0.9, 0.999), IdDict{Any,Any}())

In [244]:
Flux.train!(loss, params(mc), dataset_complete, opt_complete, cb = throttle(evalcb_complete, 2))

loss(xc_train, yc_train) = 37.61194486155767 (tracked)
loss(xc_train, yc_train) = 84.5489824610536 (tracked)
loss(xc_train, yc_train) = 10.083031390954647 (tracked)
loss(xc_train, yc_train) = 29.43335782050599 (tracked)
loss(xc_train, yc_train) = 19.581547293827107 (tracked)
loss(xc_train, yc_train) = 4.797481731047341 (tracked)
loss(xc_train, yc_train) = 17.325621924154028 (tracked)


### Check the model

### training dataset

In [280]:
train_C = mc(xc_train[:,1:10])

Tracked 1×10 Array{Float32,2}:
 3.66119  6.16632  2.09423  -4.58189  …  -15.8033  -0.0287508  -3.65051

In [248]:
yc_train[:,1:10]

1×10 Array{Float64,2}:
 6.93372  7.0001  3.70842  -2.54874  -17.8543  …  -12.7186  1.43408  0.073232

In [281]:
for i in zip(train_C', yc_train[1:10])
    println(i)
end

(3.6611907f0 (tracked), 6.93372)
(6.166321f0 (tracked), 7.0001)
(2.094234f0 (tracked), 3.70842)
(-4.581895f0 (tracked), -2.54874)
(-20.302265f0 (tracked), -17.8543)
(6.0142183f0 (tracked), 9.91444)
(-0.90044427f0 (tracked), 3.45232)
(-15.803331f0 (tracked), -12.7186)
(-0.028750844f0 (tracked), 1.43408)
(-3.6505148f0 (tracked), 0.073232)


### testing dataset

In [282]:
test_C = mc(xc_test[:,1:10])

Tracked 1×10 Array{Float32,2}:
 3.11465  -2.76543  -8.19415  …  -4.16424  -1.15869  81.4873  79.357

In [250]:
yc_test[:,1:10]

1×10 Array{Float64,2}:
 5.08376  -2.76404  -4.37406  -0.399917  …  0.840197  86.1891  83.8421

In [283]:
for i in zip(test_C', yc_test[1:10])
    println(i)
end

(3.1146507f0 (tracked), 5.08376)
(-2.765433f0 (tracked), -2.76404)
(-8.194154f0 (tracked), -4.37406)
(-2.704211f0 (tracked), -0.399917)
(-1.329688f0 (tracked), 1.61685)
(95.235214f0 (tracked), 99.06)
(-4.1642375f0 (tracked), -0.712649)
(-1.1586939f0 (tracked), 0.840197)
(81.48726f0 (tracked), 86.1891)
(79.35697f0 (tracked), 83.8421)


## The above result shows that adding more parameters in the training data could get a better model.

### create the model from scratch : model 3

In [251]:
W = param(randn(1,104)/10) 
b = param([0.]) 

Tracked 1-element Array{Float64,1}:
 0.0

In [252]:
predict(x) = W*x .+ b
meansquarederror(ŷ, y) = sum((ŷ .- y).^2)/size(y, 2)
loss(x, y) = meansquarederror(predict(x), y)
# loss(x,y) = Flux.mse(predict(x),y)

loss (generic function with 1 method)

In [254]:
η = 0.1
θ = Params([W, b])

Params([[0.11552552455746576 0.11787649584081854 … -0.032892288049661024 -0.030599694101448277] (tracked), [0.0] (tracked)])

In [255]:
predict(xc_train[:,1:10])

Tracked 1×10 Array{Float64,2}:
 140.669  99.3777  121.855  155.065  …  160.989  98.5395  102.739  154.665

In [256]:
yc_train[:,1:10]

1×10 Array{Float64,2}:
 6.93372  7.0001  3.70842  -2.54874  -17.8543  …  -12.7186  1.43408  0.073232

In [257]:
for i = 1:10
  g = gradient(() -> loss(xc_train, yc_train), θ)
  for x in θ
    update!(x, -g[x]*η)
  end
  @show loss(xc_train, yc_train)
end

loss(xc_train, yc_train) = 2.4268499601377035e13 (tracked)
loss(xc_train, yc_train) = 4.023245581459924e22 (tracked)
loss(xc_train, yc_train) = 6.6700957267711195e31 (tracked)
loss(xc_train, yc_train) = 1.1058281558472694e41 (tracked)
loss(xc_train, yc_train) = 1.8333408705593197e50 (tracked)
loss(xc_train, yc_train) = 3.03947654967295e59 (tracked)
loss(xc_train, yc_train) = 5.039116208211317e68 (tracked)
loss(xc_train, yc_train) = 8.354297769657191e77 (tracked)
loss(xc_train, yc_train) = 1.3850502417540682e87 (tracked)
loss(xc_train, yc_train) = 2.296260230453465e96 (tracked)


## From the above result, we can see that the model 3 we build from scratch is bad. The loss get bigger and bigger. The reason may be the optimizition step. 

## Conclusion:

##  It is easy to build a model by Julia Flux. Adding more parameters could improve the model accuracy and the predict value is more accurate when the target value is high. The model we build from scratch can't work as we expected. The reason may be that the model can't update the parameters correctly.